## Imports

In [1]:
import os
from PIL import Image
from glob import glob
import scipy.misc
from random import randrange
import copy
%matplotlib inline

## Helper Functions / Classes

In [7]:
class ImageGen():
    def __init__(self):
        self.base_dir = os.getcwd()
        self.data_dir = os.path.join(self.base_dir, "data")
        self.crop_dir = os.path.join(self.data_dir, "crops")
        self.bg_dir = os.path.join(self.data_dir, "bgs")
        self.gen_dir = os.path.join(self.data_dir, "gen")
        
        # crop images
        self.crop_width = 512
        self.get_crops()
        self.roomba_meter_ratio = 13.39 * 2.54 / 100
        
        # background images
        self.pixels_per_meter = 600
        self.bg_width = 1280
        self.bg_height = 720
        self.get_bgs()
        
    def get_crops(self):
        """
        reads in and resizes cropped images into the self.crop_imgs list
        """
        os.chdir(self.crop_dir)
        self.crop_filenames = sorted(glob("*"))
        self.crop_imgs = [Image.open(filename) for filename in self.crop_filenames]
        self.crop_imgs = [self.resize_img(image, self.crop_width) for image in self.crop_imgs]
        os.chdir(self.base_dir)
        
    def get_bgs(self):
        """
        reads in and resizes background images the self.bg_imgs list
        """
        os.chdir(self.bg_dir)
        self.bg_filenames = sorted(glob("*"))
        self.bg_imgs = [Image.open(filename) for filename in self.bg_filenames]
#         self.bg_imgs = [image.resize((self.bg_width, self.bg_height), resample=Image.BILINEAR)
#                         for image in self.bg_imgs]
        os.chdir(self.base_dir)
        
    def resize_img(self, img, desired_width):
        """
        resizes target img while preserving width 
        
        Args:
            img (PIL Image): the image to be resized
            desired_width: the desired width for the resized image
            
        Returns:
            resized_img (PIL Image): the resized image
        """
        width, height = img.size
        ratio = width / height
        desired_height = int(desired_width / ratio)
        resized_img = img.resize((desired_width, desired_height), resample=Image.BILINEAR)
        return resized_img
    
    def crop_center(self, img, width, height):
        left  = int((img.width / 2) - (width / 2))
        right = left + width
        upper = int((img.height / 2) - (height / 2))
        lower = upper + height
        return img.crop((left, upper, right, lower))
        
    def overlay_imgs(self, fg, bg, x, y, rot, zoom):
        """
        overlays a foreground image onto a background image
        
        Args:
            fg (PIL Image): the foreground image
            bg (PIL Image): the background image
            x (int): the x position of the upper left corner to overlay image
            y (int): the y position of the upper left corner to overlay image
            
        Returns:
            bg_copy (PIL Image): the image of foreground overlayed on background
        """
        fg_copy = copy.deepcopy(fg)
        fg_copy = self.resize_img(fg_copy, int(self.pixels_per_meter * 
                                               self.roomba_meter_ratio * 
                                               zoom))
        roomba_rot = randrange(0,360)
        fg_copy = fg_copy.rotate(roomba_rot, expand=True, resample=Image.BILINEAR)
        
        bg_copy = copy.deepcopy(bg)
        bg_copy = bg_copy.rotate(rot, expand=True, resample=Image.BILINEAR)
        bg_copy = self.crop_center(bg_copy,
                                   int(self.bg_width / zoom) + 1, # +1 to fix transparent
                                   int(self.bg_height / zoom) + 1)
        bg_copy = self.resize_img(bg_copy, self.bg_width)
        print(fg_copy.width, fg_copy.height, roomba_rot)
        try:
            x = randrange(0, 1280-fg_copy.width)
            y = randrange(0, 720-fg_copy.height)
            bg_copy.paste(fg_copy, (x, y), fg_copy)
            print((x,y), (x+fg_copy.width, y+fg_copy.height))
            return bg_copy
        except:
            return None

In [8]:
imageGen = ImageGen()

In [9]:
fg = imageGen.crop_imgs[1]
bg = imageGen.bg_imgs[2]

In [5]:
from random import randrange

In [10]:
i = 0
while i < 100:
    x = 0
    y = 0
    rot = randrange(0,360)
    height = randrange(150,900) / 300
    zoom = (1280/3600) * 3 / height
    test = imageGen.overlay_imgs(fg, bg, x, y, rot, zoom)
    if test != None:
        test.save("/tmp/test{}.png".format(i))
        i += 1

133 134 317
(829, 151) (962, 285)
359 373 12
(864, 269) (1223, 642)
90 88 80
(1051, 63) (1141, 151)
131 134 202
(381, 25) (512, 159)
211 218 162
(350, 274) (561, 492)
294 294 40
(845, 304) (1139, 598)
331 325 66
(720, 288) (1051, 613)
151 146 278
(83, 280) (234, 426)
279 276 129
(849, 298) (1128, 574)
110 110 323
(521, 48) (631, 158)
172 169 125
(348, 28) (520, 197)
229 230 222
(646, 195) (875, 425)
355 369 350
(310, 302) (665, 671)
152 152 49
(79, 491) (231, 643)
565 558 55
(483, 91) (1048, 649)
109 109 46
(159, 137) (268, 246)
205 209 30
(236, 161) (441, 370)
153 153 51
(901, 23) (1054, 176)
121 122 333
(359, 592) (480, 714)
99 97 75
(969, 305) (1068, 402)
228 223 295
(402, 458) (630, 681)
103 106 12
(1010, 351) (1113, 457)
145 141 72
(674, 156) (819, 297)
269 270 318
(265, 174) (534, 444)
155 147 275
(266, 348) (421, 495)
114 115 23
(1018, 62) (1132, 177)
124 127 340
(372, 97) (496, 224)
204 204 221
(714, 169) (918, 373)
243 241 53
(457, 1) (700, 242)
113 119 5
(829, 12) (942, 131)


In [ ]:
%%timeit
imageGen.resize_img(imageGen.crop_imgs[0], 256);

In [ ]:
test_img = test_img.rotate(randrange(0,360), expand=True)

In [ ]:
fg_copy = copy.deepcopy(imageGen.crop_imgs[1])
fg_copy.size

In [ ]:
fg_copy = fg_copy.rotate(45, expand=True)
fg_copy.size

## Get the cropped images

## Get background images

## Generate image

In [ ]:
os.chdir(CROP_DIR)



